## 3.2. Evaluación de la eficacia del modelo
En este libro practicaremos con los principales métodos para medir la eficacia o el rendimiento de nuestros modelos de aprendizaje. 

Realizaremos las distintas pruebas sobre la base de datos IRIS: 

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

# Cargamos el conjunto de datos de IRIS
X, y = datasets.load_iris(return_X_y=True)
print(X.shape, y.shape)

(150, 4) (150,)


### 3.2.1. Hold-out
La forma más sencilla de particionar nuestros datos de entrenamiento y prueba es a través de un hold-out. El método de sklearn llamado train_test_split nos proporciona esta funcionalidad. Este método recibe como parámetros los conjuntos a particionar, además del parámetro test_size con el que establece el tanto por uno de datos que formarán parte del conjunto de prueba.

In [2]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(90, 4) (90,)
(60, 4) (60,)


A continuación, entrenaremos una máquina de soporte vectorial (SVM):

In [3]:
from sklearn.svm import SVC
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)   # Nota: fijamos el valor de C=1 para este ejemplo
print(clf.score(X_test, y_test))

0.9666666666666667


Cuando se evalúan diferentes hiperparámetros para los predictores, como el hiperparámetro C de la SVM, sigue existiendo  riesgo de sobreajuste en el conjunto de prueba porque los parámetros pueden ajustarse hasta que el estimador tenga un rendimiento óptimo. De este modo, el conocimiento sobre el conjunto de pruebas puede "filtrarse" en el modelo y las métricas de evaluación ya no informan sobre el rendimiento de la generalización. Para resolver este problema, se puede reservar otra parte del conjunto de datos, a la cual se llama "conjunto de validación": el entrenamiento se lleva a cabo en el conjunto de entrenamiento, después se realiza la evaluación en el conjunto de validación y, cuando el experimento parece tener éxito, se puede realizar la evaluación final en el conjunto de prueba.

Sin embargo, al dividir los datos disponibles en tres conjuntos, reducimos drásticamente el número de muestras que pueden utilizarse para el aprendizaje del modelo, y los resultados pueden depender de una elección aleatoria concreta para el par de conjuntos (entrenamiento, validación).

### 3.2.2. Cross-validation

Una solución a este problema es un procedimiento denominado validación cruzada (CV, por sus siglas en inglés). El conjunto de prueba debe seguir utilizándose para la evaluación final, pero el conjunto de validación ya no es necesario cuando se realiza la CV. En el enfoque básico, denominado validación cruzada en $k$-pliegues ($k$-fold cross-validation), el conjunto de entrenamiento se divide en $k$ conjuntos más pequeños (más adelante se describen otros enfoques, pero en general siguen los mismos principios). Se sigue el siguiente procedimiento para cada uno de los $k$ "pliegues":

- Se entrena un modelo utilizando $k-1$ pliegues como datos de entrenamiento;

- el modelo resultante se valida con la parte restante de los datos (es decir, se utiliza como conjunto de prueba para calcular una medida de rendimiento, como la precisión).

La medida de rendimiento obtenida mediante la validación cruzada en $k$-pliegues es la media de los valores calculados en el bucle. Este método puede ser costoso desde el punto de vista computacional, pero no desperdicia demasiados datos (como ocurre cuando se fija un conjunto de validación arbitrario), lo que supone una gran ventaja en problemas en los que el número de muestras es pequeño.

#### 3.2.2.1. Función cross_val_score

La forma más sencilla de utilizar la validación cruzada es llamar a la función de ayuda cross_val_score sobre el estimador y el conjunto de datos.

El siguiente ejemplo muestra cómo estimar la precisión de una SVM con kernel lineal en el conjunto de datos iris,dividiendo los datos, ajustando un modelo y calculando la puntuación 5 veces consecutivas (con diferentes divisiones (splits) cada vez):

In [4]:
from sklearn.model_selection import cross_val_score
clf = svm.SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(clf, X, y, cv=5)
print(scores)

[0.96666667 1.         0.96666667 0.96666667 1.        ]


La puntuación media y la desviación típica vienen dadas por:

In [5]:
print("La CV obtuvo una accuracy media de {:.3f} con desviación estándar igual a {:.3f}".format(scores.mean(), scores.std()))

La CV obtuvo una accuracy media de 0.980 con desviación estándar igual a 0.016


Por defecto, la puntuación calculada en cada iteración de la CV es el método de puntuación del predictor. Es posible cambiarlo utilizando el parámetro scoring y, por ejemplo, utilizar la macro-F1:

In [6]:
from sklearn import metrics
scores = cross_val_score(clf, X, y, cv=5, scoring='f1_macro')
print(scores)

[0.96658312 1.         0.96658312 0.96658312 1.        ]


Consultar el siguiente <a href='https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter'>enlace</a> para ver el listado detallado de las métricas de evaluación que se pueden utilizar.

Se puede apreciar que los resultados utilizando la accuracy y la macro-F1 son iguales. Ello se debe a que los ejemplos están balanceados en base a su clase. Cuando el parámetro cv de la función cross_val_score es un entero, ésta utiliza la estrategia KFold o StratifiedKFold para particionar los datos. La elección depende de si el predictor deriva de la clase ClassifierMixin. Sklearn permite usar más estrategias. Para ello, basta con indicarla en el parámetro cv:  

In [7]:
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
scores = cross_val_score(clf, X, y, cv=cv)
print(scores)

[0.97777778 0.97777778 1.         0.95555556 1.        ]


Del mismo modo que es importante probar un predictor con datos que no se han tenido en cuenta en el entrenamiento, el preprocesamiento (como la estandarización, la selección de características, etc.) y otras transformaciones de datos similares deben aprenderse de un conjunto de entrenamiento y aplicarse a los datos que no se han tenido en cuenta para la predicción. Por ejemplo, el siguiente código nos muestra cómo se podría hacer este procedimiento cuando se aplica a un conjunto de entrenamiento y de prueba (en CV habría que hacer esto mismo en cada uno de los pliegues):

In [8]:
from sklearn import preprocessing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
clf = svm.SVC(C=1).fit(X_train_transformed, y_train)
X_test_transformed = scaler.transform(X_test)
score = clf.score(X_test_transformed, y_test)
print(score)

0.9333333333333333


Un Pipeline facilita la composición de estimadores, proporcionando este comportamiento bajo validación cruzada:

In [9]:
from sklearn.pipeline import make_pipeline
clf = make_pipeline(preprocessing.StandardScaler(), svm.SVC(C=1))
scores = cross_val_score(clf, X, y, cv=cv)
print(scores)

[0.97777778 0.93333333 0.95555556 0.93333333 0.97777778]


#### 3.2.2.2. Función cross_validate

Una alternativa a cross_val_score es la función cross_validate, que difiere de en dos aspectos:
- Permite especificar múltiples métricas para la evaluación.
- Devuelve un dict que contiene tiempos de ajuste, tiempos de puntuación (y opcionalmente puntuaciones de entrenamiento así como estimadores ajustados) además de la puntuación de la prueba.

Se puede establecer el parámetro return_train_score a verdadero para evaluar las puntuaciones en el conjunto de entrenamiento.
También puede conservar el estimador ajustado en cada conjunto de entrenamiento estableciendo return_estimator=True.

In [10]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
scoring = ['precision_macro', 'recall_macro']
clf = svm.SVC(kernel='linear', C=1, random_state=0)
scores = cross_validate(clf, X, y, scoring=scoring)
print(sorted(scores.keys()))
print(scores['test_recall_macro'])

['fit_time', 'score_time', 'test_precision_macro', 'test_recall_macro']
[0.96666667 1.         0.96666667 0.96666667 1.        ]


#### 3.2.2.3. Iteradores

En ocasiones trabajaremos con predictores propios o de una librería diferente a sklearn. Para estos casos, el uso de las funciones anteriores no es, en principio, aplicable. Sin embargo, sí podemos hacer uso de los mecanismos que nos proporcionan los iteradores para definir los pliegues. 

##### 3.2.2.3.1. KFold

KFold divide el conjunto de muestras en grupos, llamados pliegues, de igual tamaño (si es posible). La función de predicción se aprende utilizando $k-1$ pliegues, y el pliegue que queda fuera se utiliza para la prueba.

Ejemplo de validación cruzada de 2 pliegues en un conjunto de datos con 4 muestras:

In [11]:
from sklearn.model_selection import KFold

X = ["a", "b", "c", "d"]
kf = KFold(n_splits=2)
splits = kf.split(X)
for train, test in splits:
    print('{} {}'.format(train, test))

[2 3] [0 1]
[0 1] [2 3]


Cada pliegue está constituido por dos matrices de índices: la primera con los elementos del conjunto de entrenamiento y la segunda con los del conjunto de prueba. Así, se pueden crear los conjuntos de entrenamiento/prueba utilizando la indexación numpy:

In [12]:
for train, test in splits:
    X_train, y_train, X_test, y_test = X[train], y[train], X[test], y[test]

##### 3.2.2.3.2. Stratified KFold

StratifiedKFold es una variación de k-fold que devuelve pliegues estratificados: cada conjunto contiene aproximadamente el mismo porcentaje de muestras de cada clase objetivo que el conjunto completo.

A continuación se muestra un ejemplo de validación cruzada estratificada triple en un conjunto de datos con 50 muestras de dos clases desequilibradas. Se muestra el número de muestras de cada clase y se compara con KFold.

In [13]:
from sklearn.model_selection import StratifiedKFold
X, y = np.ones((50, 1)), np.hstack(([0] * 45, [1] * 5))

print('Stratified KFold:')
skf = StratifiedKFold(n_splits=3)
for train, test in skf.split(X, y):
    print('train -  {}   |   test -  {}'.format(np.bincount(y[train]), np.bincount(y[test])))

print('KFold:')
kf = KFold(n_splits=3)
for train, test in kf.split(X, y):
    print('train -  {}   |   test -  {}'.format(np.bincount(y[train]), np.bincount(y[test])))

Stratified KFold:
train -  [30  3]   |   test -  [15  2]
train -  [30  3]   |   test -  [15  2]
train -  [30  4]   |   test -  [15  1]
KFold:
train -  [28  5]   |   test -  [17]
train -  [28  5]   |   test -  [17]
train -  [34]   |   test -  [11  5]


##### 3.2.2.3.3. Repeated KFold

RepeatedKFold repite el K-Fold $n$ veces (idemo para StratifiedRepeatedKFold). Se puede utilizar cuando se requiere ejecutar KFold $n$ veces, produciendo diferentes divisiones en cada repetición.

Ejemplo de K-Fold repetido 2 veces:

In [14]:
from sklearn.model_selection import RepeatedKFold
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
random_state = 12883823
rkf = RepeatedKFold(n_splits=2, n_repeats=2, random_state=random_state)
for train, test in rkf.split(X):
    print('{} {}'.format(train, test))

[2 3] [0 1]
[0 1] [2 3]
[0 2] [1 3]
[1 3] [0 2]


De forma similar, el RepeatedStratifiedKFold repite el Stratified K-Fold $n$ veces con diferente aleatorización en cada repetición.

##### 3.2.2.3.4. Leave One Out 

LeaveOneOut es una validación cruzada simple. Cada conjunto de aprendizaje se crea tomando todas las muestras excepto una, siendo el conjunto de prueba la muestra dejada fuera. Así, para n muestras, tenemos n conjuntos de entrenamiento diferentes y n conjuntos de prueba diferentes.

In [15]:
from sklearn.model_selection import LeaveOneOut

X = [1, 2, 3, 4]
loo = LeaveOneOut()
for train, test in loo.split(X):
    print('{} {}'.format(train, test))

[1 2 3] [0]
[0 2 3] [1]
[0 1 3] [2]
[0 1 2] [3]


##### 3.2.2.3.5. Group k-fold

GroupKFold es una variación de $k$-fold cross-validation que garantiza que el mismo grupo no esté representado tanto en los conjuntos de prueba como en los de entrenamiento. Por ejemplo, si los datos se obtienen de diferentes sujetos con varias muestras por sujeto y si el modelo es lo suficientemente flexible como para aprender de características muy específicas de la persona, podría fallar a la hora de generalizar a nuevos sujetos. GroupKFold permite detectar este tipo de situaciones de sobreajuste.

Imagine que tiene tres sujetos, cada uno con un número asociado del 1 al 3:

In [16]:
from sklearn.model_selection import GroupKFold

X = [0.1, 0.2, 2.2, 2.4, 2.3, 4.55, 5.8, 8.8, 9, 10]
y = ["a", "b", "b", "b", "c", "c", "c", "d", "d", "d"]
groups = [1, 1, 1, 2, 2, 2, 3, 3, 3, 3]

gkf = GroupKFold(n_splits=3)
for train, test in gkf.split(X, y, groups=groups):
    print('{} {}'.format(train, test))

[0 1 2 3 4 5] [6 7 8 9]
[0 1 2 6 7 8 9] [3 4 5]
[3 4 5 6 7 8 9] [0 1 2]


Cada sujeto está en un pliegue de prueba diferente, y el mismo sujeto nunca está tanto en el de prueba como en el de entrenamiento. Observe que los pliegues no tienen exactamente el mismo tamaño debido al desequilibrio de los datos. La misma estrategia es aplicable al leave-one-out:

In [17]:
from sklearn.model_selection import LeaveOneGroupOut
logo = LeaveOneGroupOut()
for train, test in logo.split(X, y, groups=groups):
    print('{} {}'.format(train, test))

[3 4 5 6 7 8 9] [0 1 2]
[0 1 2 6 7 8 9] [3 4 5]
[0 1 2 3 4 5] [6 7 8 9]


Si las proporciones de clase deben estar equilibradas en todos los pliegues, StratifiedGroupKFold es una mejor opción:

In [18]:
from sklearn.model_selection import StratifiedGroupKFold
X = list(range(18))
y = [1] * 6 + [0] * 12
groups = [1, 2, 3, 3, 4, 4, 1, 1, 2, 2, 3, 4, 5, 5, 5, 6, 6, 6]
sgkf = StratifiedGroupKFold(n_splits=3)
for train, test in sgkf.split(X, y, groups=groups):
    print('{} {}'.format(train, test))


[ 0  2  3  4  5  6  7 10 11 15 16 17] [ 1  8  9 12 13 14]
[ 0  1  4  5  6  7  8  9 11 12 13 14] [ 2  3 10 15 16 17]
[ 1  2  3  8  9 10 12 13 14 15 16 17] [ 0  4  5  6  7 11]


#### 3.2.2.4. Validación cruzada para series temporales

Las series temporales se caracterizan por la correlación entre observaciones cercanas en el tiempo (autocorrelación). Sin embargo, las técnicas clásicas de validación cruzada, como KFold y ShuffleSplit, suponen que las muestras son independientes y se distribuyen de forma idéntica, y darían lugar a una correlación poco razonable entre las instancias de entrenamiento y de prueba (lo que produciría estimaciones deficientes del error de generalización) en los datos de series temporales. Por lo tanto, es muy importante evaluar nuestro modelo para datos de series temporales en las observaciones "futuras" menos parecidas a las que se utilizan para entrenar el modelo. 

TimeSeriesSplit es una variación de k-fold que devuelve los primeros pliegues como conjunto de entrenamiento y el tercer pliegue como conjunto de prueba. A diferencia de los métodos estándar de validación cruzada, los conjuntos de entrenamiento sucesivos son superconjuntos de los anteriores. Además, añade todos los datos sobrantes a la primera partición de entrenamiento, que siempre se utiliza para entrenar el modelo.

Ejemplo de validación cruzada de 3 particiones de series temporales en un conjunto de datos con 6 muestras:

In [19]:
from sklearn.model_selection import TimeSeriesSplit

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4, 5, 6])
tscv = TimeSeriesSplit(n_splits=3)
print(tscv)
TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None)
for train, test in tscv.split(X):
    print('{} {}'.format(train, test))

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None)
[0 1 2] [3]
[0 1 2 3] [4]
[0 1 2 3 4] [5]


### 3.2.3. Bootstrap

Este método consiste en crear, en cada una de las iteraciones del método, un conjunto de entrenamiento y otro de prueba. El conjunto de entrenamiento, del mismo tamaño que el conjunto original, se obtiene a partir de un muestreo con reemplazamiento, de forma que podrá contener elementos duplicados. El conjunto de prueba estará formado por los elementos no seleccionados del conjunto original. 


In [20]:
from sklearn.utils import resample
X = np.array([[0., 0.], [1., 1.], [2., 2.], [3., 3.], [4., 4.], [5., 5.], [6., 6.], [7., 7.]])
y = np.array([0, 1, 2, 3, 4, 5, 6, 7])

n_iterations = 10
for i in range(n_iterations):
    X_train, y_train = resample(X, y)
    # Seleccionamos para el conjunto de prueba los elementos no seleccionados en X_train
    X_test, y_test = [], []
    for i in range(len(X)): 
        if X[i] not in X_train:
            X_test.append(X[i].tolist())
            y_test.append(y[i].tolist())
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    # A partir de aquí ajustaríamos el predictor/clasificador
